In [1]:
import PIL
import subprocess
import re
import os
import time
import io
import cv2
import requests
import bs4
import jieba
import jieba.analyse

In [2]:
def capture_img():
    subprocess.call('.\\adb\\adb shell /system/bin/screencap -p /sdcard/screenshot.png')
    time.sleep(0.1)
    subprocess.call('.\\adb\\adb pull /sdcard/screenshot.png %s' % './tmp/screenshot.png')
    time.sleep(0.1)
    
    return cv2.imread('./tmp/screenshot.png')

In [3]:
%run aip_key.py

In [4]:
from aip import AipOcr

client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

In [5]:
def ocr(im, region):
    im = im[region[0]:region[1],region[2]:region[3],:]
    
    rst, byte_arr = cv2.imencode('.jpg',im,[int(cv2.IMWRITE_JPEG_QUALITY), 90])
    
    rst = client.basicGeneral(byte_arr.tobytes())

    s = ''.join([p['words'] for p in rst['words_result']])

    if s != None and s != '':
        if re.match(r"^A|B|C|\d+\.|,|。|，", s) != None:
            ss = re.split(r"\.|,|。|，",s)
            if len(ss) > 1:
                s = ''.join(ss[1:])
            
        return s
    return ''

In [6]:
def go_search(s, engine='google'):
    if engine == 'google':
        os.system('start www.google.com/search?q=%s' % s)
    elif engine == 'baidu':
        os.system('start www.baidu.com/s?wd=%s' % s)

In [7]:
def search(question):
    rst = requests.get('http://www.baidu.com/s?wd=%s' % question)
    rst.encoding = 'utf-8'
    search_rst = rst.text
    
    soup = bs4.BeautifulSoup(search_rst, 'html.parser')
    rst = soup.select('div#content_left')[0]
    return rst.text

In [8]:
def occur_count(s,words):
    n = 0
    for word in words:
        n += len(re.findall(word, s))
    return n

In [9]:
def keywords(question, opt_a, opt_b, opt_c):
    quest_words = jieba.analyse.extract_tags(question, allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    opt_words = jieba.analyse.extract_tags(' '.join([opt_a,opt_b,opt_c]), allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    words = quest_words + opt_words
    return words

In [16]:
def search_ans():
    im = capture_img()
    print('Parsing pic...')
    question = ocr(im,(300,600,0,1080))

    if question != '':
        opt_a = ocr(im,(655,815,0,1080))
        opt_b = ocr(im,(845,1005,0,1080))
        opt_c = ocr(im,(1035,1200,0,1080))

        quest_words = list(map(lambda p:p.word, filter(lambda p:p.flag.startswith('n'), 
                                                 jieba.posseg.cut(question))))

        opts = [opt_a, opt_b, opt_c]
        opt_words = []
        for opt in opts:
            opt_words.append(list(map(lambda p:p.word, filter(lambda p:p.flag.startswith('n'), 
                                                 jieba.posseg.cut(opt)))))

        rst = search(question)
        time.sleep(0.1)
        rst += search('%20'.join(quest_words))

        opt_cnt = []
        for i, opt_w in enumerate(opt_words):
            opt_cnt.append(occur_count(rst, opt_w))


        ans = [('A',opt_a,opt_cnt[0]),
              ('B',opt_b,opt_cnt[1]),
              ('C',opt_c,opt_cnt[2])]

        print('question:', question)
        for p in sorted(ans, key=lambda p:p[2]):
            print(p)

        go_search(question)

    else:
        print('No question found!')

In [17]:
while True:
    ipt = input()
    if ipt == 'q':
        break
    else:
        search_ans()


Parsing pic...
question: 毛泽东在自己的词作《沁园春雪》中没有提到哪位英雄?
('A', '嬴政1.04万', 0)
('B', '刘彻3.84万', 0)
('C', '刘邦4.98万', 0)
q
